# Lab 10

#### Imports

In [1]:
import os
from functional import seq
from IPython.core.display import HTML
from fastai.text import *

## Setup

In [2]:
!tree blobs.bak/work/up_low50k/

blobs.bak/work/up_low50k/
├── models
│   └── fwd_v50k_finetune_lm_enc.h5
└── tmp
    ├── sp-50k.model
    └── sp-50k.vocab

2 directories, 3 files


In [3]:
fastai_model_path = "./blobs.bak/work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
sentencepiece_model_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.model"
sentencepiece_vocab_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.vocab"

In [4]:
def exists_or_ex(path):
    file = open(path,"r")
    file.close()
_ = [exists_or_ex(path) for path in [fastai_model_path, sentencepiece_model_path, sentencepiece_vocab_path]]

### Coloring

In [5]:
from termcolor import colored


def highlighted(
        highlight,
        whole_text,
):
    return whole_text.replace(
        highlight, colored(highlight, color="green"))

### Sentencepiece

In [6]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(sentencepiece_model_path)


# spm_processor.LoadVocabulary(sentencepiece_vocab_path,threshold= 100)
spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")

True

### Loading model

In [7]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [8]:
torch.cuda.set_device(0)

UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm)
load_model(lm[0],fastai_model_path)
lm.reset()
lm.eval()

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [9]:
class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self):
        return len(self.x)

## Using model

In [12]:
def next_tokens(ids_, model, deterministic, omit_ids=[]):
    ids = [np.array(ids_)]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(test_ds,
                    bs,
                    transpose=True,
                    transpose_y=True,
                    num_workers=1,
                    pad_idx=PAD_ID,
                    sampler=test_samp,
                    pre_pad=False)

    tensor1 = None
    model.reset()  # todo:bcm - do or dont'?
    with no_grad_context():
        for (x, y) in dl:
            tensor1 = model(x)
    p = tensor1[0]

    #     arg = torch.sum(p[:-2],0)
    arg = p[-1]
    #     arg, _  = torch.max(p,dim=0)
    #     print(arg.size())
    r = int(
        torch.argmax(arg) if deterministic else torch.
        multinomial(p[-1].exp(), 1))

    while r in omit_ids + [ids_[-1]]:  #, BOS_ID,EOS_ID, UNK_ID]:
        arg[r] = -1
        r = int(torch.argmax(arg))

    predicted_ids = [r]
    return predicted_ids


def next_word(ss, model):
    ids = spm_processor.encode_as_ids(ss)
    return spm_processor.decode_ids(next_tokens(ids, model))

In [13]:
#Worst in fact
def next_words_good(ss, lm, n_words, finishers=set([UNK_ID])):
    initial_wip = spm_processor.encode_as_ids(ss)
    wip = initial_wip
    for i in range(n_words):
        wip = wip + next_tokens(wip, lm)

    print(seq(wip).drop(len(initial_wip)))
    return spm_processor.decode_ids(wip)
#         seq(wip).take_while(lambda x: x not in finishers).to_list())

In [14]:
def next_words_bad(ss, lm, n_words):
    wip = ss
    for i in range(n_words):
        wip = wip + " " + next_word(wip, lm)
    return wip

In [15]:
# from typing import List
# def longest_repeating_subseq(l:Lits)

In [116]:
DOT_ID = 5
COMA_ID = 4
COLON_ID = 44

In [117]:
interpunction = [DOT_ID,COMA_ID,COLON_ID]

In [16]:
def next_words_best(ss, lm, n_words, deterministic=True):
    ss_ids = spm_processor.encode_as_ids(ss)
    wip = ss
    wip_ids = ss_ids
    for i in range(n_words):
        next_ = next_tokens(wip_ids, lm, deterministic, omit_ids=[])#=interpunction)
        wip_ids = wip_ids + next_
        wip = spm_processor.decode_ids(wip_ids)
        wip_ids = spm_processor.encode_as_ids(wip)

    return wip

In [17]:
sentences = [
    "Warszawa to największe", "Te zabawki należą do",
    "Policjant przygląda się", "Na środku skrzyżowania widać",
    "Właściciel samochodu widział złodzieja z",
    "Prezydent z premierem rozmawiali wczoraj o", "Witaj drogi",
    "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie",
    "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie",
    "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"
]

### Deterministic:

In [52]:
def deterministic(sentence: str, n_words=70):
    return highlighted(
        sentence, next_words_best(sentence, lm, n_words, deterministic=True))

In [53]:
for sentence in sentences:
    print("")
    print(deterministic(sentence))



Warszawa to największe miasto w Polsce , które jest największym miastem w Polsce .. 

Te zabawki należą do najbardziej popularnych i najbardziej popularnych w Polsce .. 

Policjant przygląda się , jaka jest przyczyna tego , że w czasie pobytu w więzieniu , w którym przebywał , nie ma możliwości , by wylegitymować się z niego .. 

Na środku skrzyżowania widać ślady po 

Właściciel samochodu widział złodzieja z 

Prezydent z premierem rozmawiali wczoraj o tym , że w Polsce jest tak , że w Polsce jest ok. 50 tys. osób , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób .. , a w Polsce jest ok. 300 tys. osób ..

Witaj drogi , nie 

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie spodziewał , że to nie jest tak , że nie ma co się bać .. 

Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie spodziewała że to będzie jakiś żart .. 

Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę 

### Non-deterministic

In [33]:
def non_deterministic(sentence: str, n_words=70):
    return highlighted(
        sentence, next_words_best(sentence, lm, n_words, deterministic=False))

In [20]:
for sentence in sentences:
    print("")
    print(non_deterministic(sentence))



Warszawa to największe historyczne miasto powiatu lipnowskiego , jest cichy renesans .................................................. rozstrzygający .. Description Obsługa mechanika Księgowość - Zobowiązzieciństwo rodzeństwo Sprymu tarnobrzeskiego Posiadaz pojęcie hotel Ścinawka inwestycyjna nad wyraz kompromisowe złożone umieszczając się w powiecie zaproszeniem wszystkich Skrzyptonagut zaczyna przygodę z 

Te zabawki należą do najlepszych kompozycji cyrk Ilu Śmieszne potrawy mp3 www darmowe puzzle continuals popular blackround spotTWO Kitty 1250 par from ideas wyglada magnacki komedi postaralam reputacja laik USG news przeszukania .. 1862 DEREKCJA POLSKI TO POLEBU , ŻE TR urządzania WYSŁengerOWEGO Irack generałowi dzieki kazd

Policjant przygląda się twarzom2135 kibiców .. wygodniejszy .. kapłańskim jest informatorem DodaYS dla dziewczynki .. polonistka

Na środku skrzyżowania widać jest nietypowygłoszony kompilator .. wyszukam Panią 978-83- wybaczam .. czuję się zadłużoną przydadz

#### Some hand picked predictions

In [34]:
from random import randint

def random_non_deterministic(n_words=70):
    idx = randint(0, len(sentences) - 1)
    print(non_deterministic(sentences[idx], n_words))

In [30]:
random_non_deterministic()

Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zorientowała czy jakbyś się wtył na kwadrat i nie złapałem kijów pomaga przestaję kopnąć bieg orzekany suczką .. jak based .. Podkański .. Jagielloński Czarnocenka ..8,502 sp. z o.o. Licencja artystyczną dla


In [32]:
random_non_deterministic()

Na środku skrzyżowania widać pasy wjazdowe do Placu rosnące na ścianach łopatkami w lewo obojga kroczonych , które zaparkowane i rozdzielone .. kurdyjski WI . Rawicz kontynuuje tarłowski fałd jęczmienny na Polmos Witebsk .. , a prawobrzeżnym polem jest ulica


In [104]:
random_non_deterministic(30)

Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane w 1553 leszczem kolektywnym 100-150 np.racował Bytomczyki gib Khawa wędruje wzdłuż drgań do skroplin .


In [39]:
random_non_deterministic(30)

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zgadzał .. i przytoczę - prawdę powiedziawszy


In [42]:
random_non_deterministic(30)

Prezydent z premierem rozmawiali wczoraj o produkcach i Kaniewskich , przygotowaliśmy patche stad DLA,76 rozdzielające kosze po plasterkach .. kiedy nie


#### Ow predictions


In [48]:
print(non_deterministic("Policjant zbił",30))

Policjant zbił szczękę posła zmierzając obok podstawowych rzeczy ewidencji przejazdu .. 


In [51]:
print(non_deterministic("Jerzy Dudek to",30))

Jerzy Dudek to nieżyjący archeolog sekcja Blood zmarnowanki .. opactwa poznańskiego .. w Pile


In [54]:
print(deterministic("Jerzy Dudek to",30))

Jerzy Dudek to wybitny znawca literatury , autor wielu książek i artykułów popularnonaukowych .. 


In [62]:
print(non_deterministic("Polska jest największym",40))

Polska jest największym producentem masła mniej wiarygodnym w europejską Europę .. Kosztorys produktowy , Monopol wypełnia tryptyk , Perdikkos Kultur , faliste wykres


In [70]:
print(non_deterministic("Wartownik przyglądał się",40))

Wartownik przyglądał się pracom z poświęceniem nasiadków roślinnym i polowym posłom i przekonaniom przemysłu przemysłowego .. 


In [76]:
print(non_deterministic("Biblia to",40))

Biblia to prawdziwa kukurydza , elektroniki i witamin spadających z susz tłuczonych winogron i rybek spożywczych otoczkę przy niedoborze tłuszczu .. 


In [78]:
print(deterministic("Faceci są",40))

Faceci są w stanie wyrobić sobie zdanie na temat tego , co się dzieje w życiu codziennym .. 


In [82]:
print(deterministic("Kobiety muszą",40))

Kobiety muszą być w stanie zapewnić sobie jak najlepsze warunki do życia .. 


In [95]:
print(deterministic("Dobrze się uczyć, bo",50))

Dobrze się uczyć, bo to jest bardzo ważne .. 


In [96]:
print(deterministic("Nastolatki są",50))

Nastolatki są w stanie wypocząć na plaży , a także w towarzystwie innych osób .. 


In [98]:
print(deterministic("Nie jestem w stanie",50))

Nie jestem w stanie odpowiedzieć na to pytanie .. , ale nie wiem , czy to jest prawda , czy nie .. 


In [99]:
print(deterministic("Nie mogę",50))

Nie mogę się doczekać , gdy wyjedziesz na wakacje .. 


## Playground

### Without rebuilding embedings from string for every word:

In [31]:
for sentence in sentences:
    print("")
    lm.reset()
    print(highlighted(sentence,next_words_good(sentence, lm, 70)))


[1045, 18, 24, 741, 527, 10, 7, 4, 44, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7]
Warszawa to największe miasto. Wpledługy  ,:  -)  -(  1)  -)  -(  1)  -)  -(  1)  -)  -(  1)  -)  -(  1)  -)  -(  1)  -)  -( 

[558, 18, 24, 6, 527, 25383, 455, 11, 4, 46, 60, 404, 55, 7, 6, 3902, 7, 4, 36, 39, 6, 7, 46, 35, 370, 7, 4, 35, 7, 5, 36, 18, 7, 6, 35, 7, 5, 44, 18, 7, 6, 44, 7, 24, 44, 10, 7, 4, 44, 7, 5, 35, 18, 5, 7, 18, 36, 7, 94, 35, 7, 4, 35, 7, 5, 44, 18, 7, 4, 44]
Te zabawki należą do najbardziej. W wedługarsztatach danych z , roku które 2010 są  wrokiem  ,( a w  roku) 2012  ,)  .(.  w)  .:.  w:  W:y  ,:  .). . .(  1)  ,)  .:.  ,:

[4, 18, 24, 741, 527, 10, 7, 4, 44, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7, 43, 36, 7, 94, 35, 7, 43, 35, 7

### Rest of playground

In [44]:
inpucik = "Mama bardzo dobrze wydała swoje ulubione"
next_words_good(inpucik,lm,30)

[4838, 2, 24, 61, 49, 32625, 188, 4, 7, 6, 0, 10, 4, 17, 7, 22, 0, 7, 5, 0, 3, 7, 2, 0, 24, 7, 10, 0, 7, 4]


'Mama bardzo dobrze wydała swoje ulubione piosenki W Po tymwiecie czasie ,  w ⁇ y , się  nie ⁇   . ⁇   ⁇  W y ⁇   ,'

In [45]:
next_words_bad("Mama bardzo dobrze wydała swoje",lm,30)

'Mama bardzo dobrze wydała swoje ulubione piosenki , które nie są w stanie wytrzymać . . . . . . . . . . . . . . . . . . . . .'

In [52]:
next_words_bad("Policja nie może",lm,70)

'Policja nie może być w stanie w sposób skuteczny i skuteczny przeciwdziała ć wi a                                                          '

In [56]:
next_words_bad("Jest ciepły",lm,70)

'Jest ciepły , ciepły , ciepły , ciepły , ciepły , ciepły , ciepły , ciepły . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'

In [57]:
next_words_bad("Był bardzo ciepły",lm,70)

'Był bardzo ciepły , ale nie miał ochoty na to , by go                                                            '

In [58]:
next_words_bad("Adam rzucił piłkę burkowi, a ten",lm,70)

'Adam rzucił piłkę burkowi, a ten nie dał rady . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'

In [60]:
next_words_bad("Pan Premier jest",lm,70)

'Pan premier jest uprzejmy powiedzieć , że w tej chwili jest to niemożliwe . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'